Необходимо написать скрипт для парсинга диалогов из файла test_data.csv. 
Получившийся скрипт необходимо выложить в гит репозиторий и прислать ссылку в качестве результата прохождения тестового задания. Данные выкладывать в гит не следует. 
Главные задачи, которые должен выполнять скрипт:
- Извлекать реплики с приветствием – где менеджер поздоровался GREETING. 
- Извлекать реплики, где менеджер представил себя INTRODUCE. 
- Извлекать имя менеджера MANAGER_NAME. 
- Извлекать название компании COMPANY_NAME. 
- Извлекать реплики, где менеджер попрощался GOODBYE.

Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

Рекомендации:
- Сделать локальную копию файла test_data.csv, в исходнике никакие данные не менять!
- Можно создать дополнительное поле в таблице test_data.csv, куда будет сохраняться результат парсинга – например, напротив реплики в столбце “insight” можно ставить флаг того, что эта реплика с приветствием greeting=True.
- Для выполнения задачи можно использовать любые библиотеки и NLP модели. 
- Попробуйте учесть возможные синонимичные выражения, которые могут помочь с извлечением данных сущностей. 


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('test_data.csv')

In [3]:
print(df.columns)
print(df.shape)

Index(['dlg_id', 'line_n', 'role', 'text'], dtype='object')
(480, 4)


In [4]:
df

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага
...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...
477,5,140,client,Спасибо спасибо
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте


### Буду использовать yargy парсер

In [5]:
!pip install natasha

In [6]:
from yargy import rule, Parser, or_
from yargy.predicates import eq, type as yargy_type, normalized, in_, gram, dictionary
from yargy.pipelines import morph_pipeline

In [7]:
# за приветствие и прощание буду принимать только корректные для деловой беседы формулировки
# для простой задачи достаточно morph_pipeline, расписать варианты относительно недолго
GREETING = morph_pipeline([
    'здравствуйте',
    'добрый день',
    'день добрый',
    'добрый вечер',
    'вечер добрый',
    'доброе утро',
    'приветствую'
    ])

GOODBYE = morph_pipeline([
    'до свидания', 
    'всего доброго',
    'хорошего вечера',
    'хорошего дня',
    'всего доброго', 
    'прощайте',
])

In [8]:
# имя следует за "зовут" чаще всего, особенно если в деловой беседе (хотя в одной представлении эти токены переставлены)
MANAGER_NAME = rule(
        gram('NOUN').optional(),
        eq('зовут'),
        gram('NOUN').optional()
)

# название компании может следовать ЗА (MANAGER_NAME) или может идти ПЕРЕД токенами "меня" и "зовут"
# впрочем, может быть вот так: "компания диджитал бизнес, ангелина"
COMPANY_NAME = rule(normalized('компания'),
        gram('NOUN'),
        gram('NOUN').optional()
)

# после получения имени и компании проще найти реплики с полным представлением менеджера
INTRODUCE = rule(
    eq('меня').optional(),
    normalized('зовут').optional(),
    MANAGER_NAME,
    normalized('компания').optional(), # из(от) компаниИ, компаниЯ, (представляю) компаниЮ и т.д.
    COMPANY_NAME   
)

In [9]:
text = []
for line in df['text']:
    text.append(line)
    
#text

In [10]:
# представление по имени и названию компании
parser = Parser(INTRODUCE)

for match in parser.findall(str(text)):
    print([x.value for x in match.tokens])

['зовут', 'ангелина', 'компания', 'диджитал', 'бизнес']
['зовут', 'ангелина', 'компания', 'диджитал', 'бизнес']
['зовут', 'ангелина', 'компания', 'диджитал', 'бизнес']
['меня', 'максим', 'зовут', 'компания', 'китобизнес']


In [11]:
# приветствие
parser = Parser(GREETING)

for match in parser.findall(str(text)):
    print([x.value for x in match.tokens])

['здравствуйте']
['Добрый', 'день']
['здравствуйте']
['здравствуйте']
['Здравствуйте']
['здравствуйте']
['Добрый', 'день']
['добрый', 'день']
['здравствуйте']


In [12]:
# прощание
parser = Parser(GOODBYE)

for match in parser.findall(str(text)):
    print([x.value for x in match.tokens])

['до', 'свидания']
['До', 'свидания']
['всего', 'доброго']
['до', 'свидания']
['до', 'свидания']
['до', 'свидания']
['до', 'свидания']
['хорошего', 'вечера']


In [13]:
# имя менеджера
parser = Parser(MANAGER_NAME)

for match in parser.findall(str(text)):
    print([x.value for x in match.tokens])

['зовут', 'ангелина']
['зовут', 'ангелина']
['зовут', 'ангелина']
['максим', 'зовут', 'компания']


====================================================================================

Здесь я на ходу переобулся и решил использовать регулярные выражения.

Оставил строчки с yargy, чтобы показать, что с ним ознакомился. 

И по крайней мере не буду дублировать GOODBYE и GREETING (тем более что их парсить легче с morph_pipeline); будет гибридный хаотичный парсинг

In [79]:
import re
import csv
import numpy as np
import pandas as pd

def find_name(text):
    # example = 'Меня зовут ангелина компания диджитал бизнес'
    pattern = r"зовут(.+)компания"
    result = re.findall(pattern, text)
    if result == [' ']:
        pattern = r"меня(.+)зовут"
        result = re.findall(pattern, text)
    return result


def find_company_name(text):
    # example = 'Меня зовут ангелина компания диджитал бизнес'
    pattern = r"компания\s(.+бизнес+)"
    result = re.findall(pattern, text)
    return result

In [100]:
with open('./test_data.csv', 'r', encoding='UTF-8') as f:
    data = csv.reader(f, delimiter=',')
    
    names = []
    company_names = []
    
    introducing = np.zeros(480, dtype=np.int64)
    
    ind = 0
    for line in data:
        mng_name = find_name(line[3])
        cmpn_name = find_company_name(line[3])
        ind += 1
        if len(mng_name) > 0 and line[2] == 'manager':   # поиск совпадений по имени менеджера
            names.append(mng_name)
            
            # в этой же реплике логично искать название компании, если менеджер представляется правильно
            cmpn_name = find_company_name(line[3])
            company_names.append(cmpn_name)
            
            introducing[ind-2] = 1     # флаг где менеджер представился
    
    
    
    print(names)
    print(company_names)   

[[' ангелина '], [' ангелина '], [' ангелина '], [' максим ']]
[['диджитал бизнес'], ['диджитал бизнес'], ['диджитал бизнес'], ['китобизнес']]


In [101]:
# теперь можно добавить флаг с представлением менеджера в новый csv
df = pd.read_csv('./test_data.csv')
df.loc[:, 'introduced'] = introducing

In [104]:
df[df['introduced'] == 1 ]

,dlg_id,line_n,role,text,introduced
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,1
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес з...,1
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес з...,1
251,3,2,manager,Добрый меня максим зовут компания китобизнес у...,1


In [113]:
df.to_csv('new_test_data.csv', sep='|')  